<a href="https://colab.research.google.com/github/PranavPhanindra/Deep-Learning-PyTorch/blob/main/23357_MTCS_205P_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries & Mounts

In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import torch
import os

from torch import nn
from torchsummary import summary
from plotly.subplots import make_subplots

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Model

## Tanh activation


In [3]:
# Plot layer-wise histograms
def plot_histogram(layer_name, activations, save_directory, filename):
    plt.hist(activations, bins=50, alpha=0.5, label=layer_name)
    plt.legend()
    plt.title("Histogram of " + layer_name + " Activations")

    # Save the figure
    save_path = os.path.join(save_directory, filename)
    plt.savefig(save_path)
    plt.close()  # Close the figure to release resources


In [4]:
class ExperimentNetTanh(nn.Module):
    def __init__(self):
        super().__init__()

        """
        Input layer with 1000 inputs , fully connected layer 1 with 500 neurons
        Each inner layer is fully connected with 500 neurons

        To initialise weights of each of the layer with a Gaussian distribution of mean 0 and standard deviation 0.01
        """
        self.Layer1 = nn.Linear(1000,500)
        nn.init.normal_(self.Layer1.weight, mean=0, std=1)

        self.Layer2 = nn.Linear(500,500)
        nn.init.normal_(self.Layer2.weight, mean=0, std=1)

        self.Layer3 = nn.Linear(500,500)
        nn.init.normal_(self.Layer3.weight, mean=0, std=1)

        self.Layer4 = nn.Linear(500,500)
        nn.init.normal_(self.Layer4.weight, mean=0, std=1)

        self.linear_tanh_stack = nn.Sequential(
            self.Layer1,
            nn.Tanh(),
            self.Layer2,
            nn.Tanh(),
            self.Layer3,
            nn.Tanh(),
            self.Layer4,
            nn.Tanh(),
        )

    def forward(self, x):
        logits = self.linear_tanh_stack(x)
        return logits

activationTanh = {}
def get_activation(name):
  def hook(model, input, output):
      activationTanh[name] = output.detach()
  return hook

myExperimentModel1 = ExperimentNetTanh()
summary(myExperimentModel1, (1000,))
print(myExperimentModel1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 500]         500,500
            Linear-2                  [-1, 500]         500,500
              Tanh-3                  [-1, 500]               0
            Linear-4                  [-1, 500]         250,500
            Linear-5                  [-1, 500]         250,500
              Tanh-6                  [-1, 500]               0
            Linear-7                  [-1, 500]         250,500
            Linear-8                  [-1, 500]         250,500
              Tanh-9                  [-1, 500]               0
           Linear-10                  [-1, 500]         250,500
           Linear-11                  [-1, 500]         250,500
             Tanh-12                  [-1, 500]               0
Total params: 2,504,000
Trainable params: 2,504,000
Non-trainable params: 0
---------------------------

In [5]:
inputTensor = torch.randn(1000)
#inputTensor

In [6]:
"""for linParams in myExperimentModel.linear_tanh_stack.named_parameters():
  print(linParams)
  print(linParams[1].size())"""

'for linParams in myExperimentModel.linear_tanh_stack.named_parameters():\n  print(linParams)\n  print(linParams[1].size())'

In [7]:
myExperimentModel1.Layer1.register_forward_hook(get_activation('Layer1'))
myExperimentModel1.Layer2.register_forward_hook(get_activation('Layer2'))
myExperimentModel1.Layer3.register_forward_hook(get_activation('Layer3'))
myExperimentModel1.Layer4.register_forward_hook(get_activation('Layer4'))
output = myExperimentModel1(inputTensor)
activationTanh['inputTensor']=inputTensor
activationTanh

{'Layer1': tensor([-6.2192e+01, -4.1790e+01,  2.9467e+01, -4.0312e+01, -1.8982e+01,
         -1.1243e+01,  2.6725e+01, -5.0403e+01,  3.6569e+01,  3.0666e+01,
          1.5822e+01,  3.7980e+01, -1.0009e+01,  1.8434e+01, -5.6110e+01,
         -2.5605e+01, -6.9899e+01, -5.3256e+01,  7.6889e+00,  2.2861e+01,
         -2.6825e+01,  1.0080e+01,  1.6712e+01,  2.9973e+01,  1.3942e+01,
          1.6603e-01, -2.1799e+00,  3.3661e+01,  1.7298e+01, -1.3828e+01,
         -8.1265e+00,  1.9699e+01, -2.7109e+01, -3.2936e+01, -8.3652e+00,
          2.2506e+00, -2.2088e+01, -2.7218e+01,  3.0066e+01, -9.2640e-01,
          1.7309e+01,  4.0383e+01,  6.2304e+01,  4.9127e+00, -1.8902e+01,
         -1.5046e+01,  1.9263e+01,  5.0379e+01, -2.1841e+01,  2.8014e+01,
         -5.1818e+01, -6.4148e+00,  1.6599e+01, -6.5194e+01,  2.1216e+01,
         -1.1121e+01, -2.8302e+00,  4.2489e+01, -2.5931e+01,  5.6897e+00,
          5.8890e-01,  6.9340e+01,  1.1104e+01,  8.6609e+01,  6.8273e+00,
         -2.3974e+01, -1.014

In [8]:
saveDir = "drive/MyDrive/Colab Notebooks/MTCS-205/Assignments/Assignment-2/tanhActivations/Exp2"
for layer in activationTanh.keys():
  plot_histogram(layer, activationTanh[layer], saveDir, layer)

In [9]:
fig = make_subplots(rows=1, cols=1)

for key, values in activationTanh.items():
    histogram = go.Histogram(x=values, opacity=0.5, name=key)
    fig.add_trace(histogram)

fig.update_layout(
    title='Tanh Activations',
    xaxis_title='Values',
    yaxis_title='Frequency',
    template='plotly_dark',  # Change the theme here
)
fig.show()
fig.write_html(saveDir + "/Exp2.html")

## Sigmoid Activation

In [10]:
class ExperimentNetSigmoid(nn.Module):
    def __init__(self):
        super().__init__()

        """
        Input layer with 1000 inputs , fully connected layer 1 with 500 neurons
        Each inner layer is fully connected with 500 neurons

        To initialise weights of each of the layer with a Gaussian distribution of mean 0 and standard deviation 0.01
        """
        self.Layer1 = nn.Linear(1000,500)
        nn.init.normal_(self.Layer1.weight, mean=0, std=1)

        self.Layer2 = nn.Linear(500,500)
        nn.init.normal_(self.Layer2.weight, mean=0, std=1)

        self.Layer3 = nn.Linear(500,500)
        nn.init.normal_(self.Layer3.weight, mean=0, std=1)

        self.Layer4 = nn.Linear(500,500)
        nn.init.normal_(self.Layer4.weight, mean=0, std=1)

        self.linear_ReLU_stack = nn.Sequential(
            self.Layer1,
            nn.Sigmoid(),
            self.Layer2,
            nn.Sigmoid(),
            self.Layer3,
            nn.Sigmoid(),
            self.Layer4,
            nn.Sigmoid(),
        )

    def forward(self, x):
        logits = self.linear_ReLU_stack(x)
        return logits

activationSigmoid = {}
def get_activation(name):
    def hook(model, input, output):
        activationSigmoid[name] = output.detach()
    return hook

myExperimentModel2 = ExperimentNetSigmoid()
summary(myExperimentModel2, (1000,))
print(myExperimentModel2)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 500]         500,500
            Linear-2                  [-1, 500]         500,500
           Sigmoid-3                  [-1, 500]               0
            Linear-4                  [-1, 500]         250,500
            Linear-5                  [-1, 500]         250,500
           Sigmoid-6                  [-1, 500]               0
            Linear-7                  [-1, 500]         250,500
            Linear-8                  [-1, 500]         250,500
           Sigmoid-9                  [-1, 500]               0
           Linear-10                  [-1, 500]         250,500
           Linear-11                  [-1, 500]         250,500
          Sigmoid-12                  [-1, 500]               0
Total params: 2,504,000
Trainable params: 2,504,000
Non-trainable params: 0
---------------------------

In [11]:
myExperimentModel2.Layer1.register_forward_hook(get_activation('Layer1'))
myExperimentModel2.Layer2.register_forward_hook(get_activation('Layer2'))
myExperimentModel2.Layer3.register_forward_hook(get_activation('Layer3'))
myExperimentModel2.Layer4.register_forward_hook(get_activation('Layer4'))
outputSigmoid = myExperimentModel2(inputTensor)
activationSigmoid['inputTensor'] = inputTensor
activationSigmoid

{'Layer1': tensor([  53.3647,  -14.3124,    3.7988,   75.2385,   12.0880,  -30.5274,
           -0.7822,  -38.5548,  -12.3837,   20.3065,   27.0192,    2.0898,
           53.5583,   16.3691,   43.5376,  -24.4204,   -4.0515,  -30.5121,
           -0.5457,  -27.7993,   24.7639,   53.8046,  -58.0623,   -2.4113,
           14.1349,  -60.9221,   20.3237,  -17.0936,   52.5820,  -17.5776,
            8.5999,    1.7878,    7.9391,  -22.3486,    0.9767,  -11.4041,
          -55.5769,   49.6932,   17.2019,  -44.4684,  -25.2999,  -19.6454,
            7.3802,    1.7674,   35.3678,   38.1839,   34.8311,  -20.8144,
          -27.1482,  -23.7470,   23.9295,   13.9336,  -23.0774,    4.1529,
          -46.8319,   33.3783,  -18.5950,  -30.9415,  -42.2308,   -7.9002,
          -68.1557,  -18.6766,   26.0441,   21.3257,   -9.9724,  -11.9326,
           -9.8908,   20.4148,   -1.9272,   16.0769,   31.9118,   -2.0226,
           16.7309,   20.4972,   77.1074,   32.9280,   35.9439,  -19.3943,
          -40.8

In [12]:
saveDir = "drive/MyDrive/Colab Notebooks/MTCS-205/Assignments/Assignment-2/sigmoidActivations/Exp2"
for layer in activationSigmoid.keys():
  plot_histogram(layer, activationSigmoid[layer], saveDir, layer)

In [13]:
fig = make_subplots(rows=1, cols=1)

for key, values in activationSigmoid.items():
    histogram = go.Histogram(x=values, opacity=0.5, name=key)
    fig.add_trace(histogram)

fig.update_layout(
    title='Sigmoid Activations',
    xaxis_title='Values',
    yaxis_title='Frequency',
    template='plotly_dark',  # Change the theme here
)
fig.show()
fig.write_html(saveDir + "/Exp2.html")

## ReLU Activation

In [14]:
class ExperimentNetReLU(nn.Module):
    def __init__(self):
        super().__init__()

        """
        Input layer with 1000 inputs , fully connected layer 1 with 500 neurons
        Each inner layer is fully connected with 500 neurons

        To initialise weights of each of the layer with a Gaussian distribution of mean 0 and standard deviation 0.01
        """
        self.Layer1 = nn.Linear(1000,500)
        nn.init.normal_(self.Layer1.weight, mean=0, std=1)

        self.Layer2 = nn.Linear(500,500)
        nn.init.normal_(self.Layer2.weight, mean=0, std=1)

        self.Layer3 = nn.Linear(500,500)
        nn.init.normal_(self.Layer3.weight, mean=0, std=1)

        self.Layer4 = nn.Linear(500,500)
        nn.init.normal_(self.Layer4.weight, mean=0, std=1)

        self.linear_Sigmoid_stack = nn.Sequential(
            self.Layer1,
            nn.ReLU(),
            self.Layer2,
            nn.ReLU(),
            self.Layer3,
            nn.ReLU(),
            self.Layer4,
            nn.ReLU(),
        )

    def forward(self, x):
        logits = self.linear_Sigmoid_stack(x)
        return logits

activationReLU = {}
def get_activation(name):
    def hook(model, input, output):
        activationReLU[name] = output.detach()
    return hook

myExperimentModel3 = ExperimentNetReLU()
summary(myExperimentModel3, (1000,))
print(myExperimentModel3)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 500]         500,500
            Linear-2                  [-1, 500]         500,500
              ReLU-3                  [-1, 500]               0
            Linear-4                  [-1, 500]         250,500
            Linear-5                  [-1, 500]         250,500
              ReLU-6                  [-1, 500]               0
            Linear-7                  [-1, 500]         250,500
            Linear-8                  [-1, 500]         250,500
              ReLU-9                  [-1, 500]               0
           Linear-10                  [-1, 500]         250,500
           Linear-11                  [-1, 500]         250,500
             ReLU-12                  [-1, 500]               0
Total params: 2,504,000
Trainable params: 2,504,000
Non-trainable params: 0
---------------------------

In [15]:
myExperimentModel3.Layer1.register_forward_hook(get_activation('Layer1'))
myExperimentModel3.Layer2.register_forward_hook(get_activation('Layer2'))
myExperimentModel3.Layer3.register_forward_hook(get_activation('Layer3'))
myExperimentModel3.Layer4.register_forward_hook(get_activation('Layer4'))
outputReLU = myExperimentModel3(inputTensor)
activationReLU['inputTensor'] = inputTensor
activationReLU

{'Layer1': tensor([-10.2734,   1.2313,  -8.2764, -10.5692, -27.8749, -34.4283,   6.2346,
         -27.1221,  49.6269, -47.3045, -22.8967, -24.1837, -50.5798, -21.3884,
           6.0380,  36.6223,  16.8037, -15.8654,  45.6153, -76.5742,   8.3334,
          10.2874,  -6.4713,  11.2357,   0.5825,  29.9045,  29.2171, -24.1476,
         -11.2290,  16.0716,  28.2684,  41.2335,  21.1839,  -6.9152, -24.7689,
           9.4895, -34.5694,  16.5531,   2.5049,   1.0051, -50.8780,  19.0201,
          -4.2953,  -9.9190,  20.5303, -22.8225, -13.4415,  59.6482,  43.0016,
          -6.9453, -58.3430, -31.4294,  -7.0501, -43.1690,  40.7965, -11.0031,
          -1.2004,   5.6823, -24.7833, -47.4285, -37.8912,  12.0907,  71.0072,
          55.8807,   9.9777, -22.8067, -40.5518, -47.7247,  43.8984,  -0.4177,
          27.2313,  -4.4582,   5.9298,  -2.2241,  12.4995,   4.7975,  -3.5158,
          47.9370,  27.4180,  -4.2880, -71.1863,  -9.3853,  10.6242,  82.3240,
         -25.2268,  22.6484, -23.4200,  58

In [16]:
saveDir = "drive/MyDrive/Colab Notebooks/MTCS-205/Assignments/Assignment-2/ReLUActivations/Exp2"
for layer in activationReLU.keys():
  plot_histogram(layer, activationReLU[layer], saveDir, layer)

In [17]:
fig = make_subplots(rows=1, cols=1)

for key, values in activationReLU.items():
    histogram = go.Histogram(x=values, opacity=0.5, name=key)
    fig.add_trace(histogram)

fig.update_layout(
    title='ReLU Activations',
    xaxis_title='Values',
    yaxis_title='Frequency',
    template='plotly_dark',  # Change the theme here
)
fig.show()
fig.write_html(saveDir + "/Exp2.html")